In [ ]:
import keras as k
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import CuDNNGRU , CuDNNLSTM , Bidirectional , GRU
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

In [ ]:
file = open("glove.twitter.27B.100d.txt")
num_vocab = 0
word_to_vec = {}
for line in file.readlines():
  
    row = line.strip().split(' ')
    word = row[0]
    vec = row[1:] 
    num_vocab += 1
    word_to_vec[word] = vec
      
print("Glove preprocessed")

file.close()

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [ ]:
X_train = train["comment_text"].fillna("Nothing").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("Nothing").values


In [ ]:
tokenizer = Tokenizer(num_words = num_vocab)

In [ ]:
tokenizer.fit_on_texts(list(X_train) + list(X_test))

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
maxlen = 100

X_train = pad_sequences(X_train , maxlen)
X_test = pad_sequences(X_test , maxlen)

In [ ]:
word_index = tokenizer.word_index

In [ ]:
min_words = min(len(word_index) , num_vocab)
print(min_words)

In [ ]:
embedding_matrix = np.random.randn(min_words , 100) * 0.001

In [ ]:
added_words = 0
for word , i in word_index.items():
      if i >= min_words:
        continue
    vector = word_to_vec.get(word)
    if vector:
        added_words+=1
        embedding_matrix[i] = vector

In [ ]:
added_words

In [ ]:
X_train , X_val , y_train , y_val = train_test_split(X_train , y_train , train_size = 0.95 , random_state = 0)

In [ ]:
class RocEval(Callback):
    def __init__(self , validation_data = () , interval = 1):
        super(Callback , self).__init__()
        self.interval = interval
        self.X_val , self.y_val = validation_data
    
    def on_epoch_end(self , epoch , logs = {}):
        y_pred = self.model.predict(self.X_val, verbose=0)
        score = roc_auc_score(self.y_val, y_pred)
        print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [ ]:
del tokenizer
del word_to_vec

In [ ]:
def model(input_shape):
    inp = k.layers.Input(shape = input_shape)
    X = k.layers.Embedding(input_dim = min_words , output_dim = 100 , weights = [embedding_matrix])(inp)
    X = k.layers.SpatialDropout1D(0.25)(X)
    X = Bidirectional(GRU(50 , return_sequences = False , recurrent_dropout = 0.25))(X)
    X = k.layers.Dense(6 , activation = 'sigmoid')(X)
  
    return k.Model(inputs = inp , outputs = X)
  

In [ ]:
toxic_model = model((100,))

In [ ]:
toxic_model.summary()

In [ ]:
toxic_model.compile(optimizer = 'rmsprop' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [ ]:
RocAuc = RocEval(validation_data=(X_val, y_val), interval=1)

In [ ]:
def Roc_score(y_true , y_pred):
  return roc_auc_score(y_true , y_pred)

In [ ]:
saver = k.callbacks.ModelCheckpoint("my_model.h5", monitor='val_loss', verbose=2, save_best_only=True)

In [ ]:
toxic_model.fit(X_train, y_train, batch_size=1024, epochs=10, validation_data=(X_val, y_val),
                 callbacks=[RocAuc , saver])

In [ ]:
prediction1 = toxic_model.predict(X_test , batch_size = 1024 , verbose = 1)

In [ ]:
best_model = load_model("my_model.h5")
prediction2 = best_model.predict(X_test , batch_size = 1024 , verbose = 1)

In [ ]:
prediction = (prediction1 + prediction2) / 2

In [ ]:
prediction = prediction.argmax(1)

In [ ]:
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = prediction
submission.to_csv("submission.csv" , index = False)